In [1]:
# from vertexai.preview.language_models import TextGenerationModel


# def interview(temperature: float = 0.2) -> None:
#     """Ideation example with a Large Language Model"""

#     # TODO developer - override these parameters as needed:
#     parameters = {
#         "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
#         "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
#         "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
#         "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
#     }

#     model = TextGenerationModel.from_pretrained("text-bison@001")
#     response = model.predict(
#         "Give me ten interview questions for the role of program manager.",
#         **parameters,
#     )
#     print(f"Response from Model: {response.text}")

# interview()

In [ ]:
# !gcloud config list
# !pip install urllib3==1.22
# !pip install urllib3==1.24.3

In [3]:
# !gcloud init
# !pip install google-cloud-aiplatform

In [4]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

warnings.filterwarnings("ignore")

In [5]:
# !pip install google-cloud-aiplatform==1.25.0
# !pip install langchain==0.0.187
# !pip install xmltodict==0.13.0
# !pip install unstructured==0.7.0 # used by langchain
# !pip install pdf2image==1.16.3 #used by langchain
# !pip install requests==2.31.0
# !pip install beautifulsoup4==4.12.2
# !pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
file = 'OutdoorClothingCatalog_1000.csv'
loader1 = CSVLoader(file_path=file)
pdfFile = '/OfferCreationviaGrumbles.pdf'

loader1
# loader = PyPDFLoader(pdfFile)
# # /Users/princekumar.k/Documents/Mylearnings/GenAI/venv/env
# pages = loader.load_and_split()
# len(pages)
# # pages[0]
# documents = pages

In [6]:
# import pathlib
# desktop = pathlib.Path(".")
# desktop.parent

In [7]:
import pathlib
desktop = pathlib.Path("./SOP")
files = []
list(desktop.iterdir())
for item in list(desktop.iterdir()):
    print(str(item))
    fileLoader = PyPDFLoader(str(item))
    pages = fileLoader.load_and_split()
    files.append(fileLoader)
    break

documents = files
documents

print(len(pages))

SOP/Incentivised Attach SOP.pdf
2


In [ ]:
import pathlib
desktop = pathlib.Path("./SOP")
files = []
list(desktop.iterdir())
pages1 = []
# for item in list(desktop.iterdir())[:1]:
for item in list(desktop.iterdir()):
    try:
        print(str(item))
        fileLoader = PyPDFLoader(str(item))
#         pages = fileLoader.load_and_split()
#         print(len(fileLoader.load_and_split()))
        pages1.append(fileLoader.load_and_split())
#         files.append(pages)
    except:
        print("An exception occurred")
  
print (type(pages1))


In [ ]:

pages = []
print(len(pages1))
for item in pages1:
    try:
        if(item.page_content):
            pages.append(item)
            print(item)

    except:
        for item2 in item:
            pages.append(item2)
print(len(pages))

In [ ]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [ ]:
context = "\n".join(str(p.page_content) for p in pages)
print("The total words in the context: ", len(context))

In [ ]:
question = "What is Incentivised Attach?"
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

In [ ]:
stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

In [ ]:
stuff_answer = stuff_chain(
    {"input_documents": pages, "question": question}, 
    return_only_outputs=True
)

In [ ]:
print(len(stuff_answer))

In [ ]:
try:
    print(
        stuff_chain(
            {"input_documents": pages[7:], "question": question},
            return_only_outputs=True,
        )
    )
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

In [ ]:
question_prompt_template = f"""
                    Answer the question as precise as possible using the provided context. \n\n
                    Context: \n {context} \n
                    Question: \n {question} \n
                    Answer:
                    """
question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

# summaries is required. a bit confusing.
combine_prompt_template = """Given the extracted content and the question, create a final answer.
If the answer is not contained in the context, say "answer not available in context. \n\n
Summaries: \n {summaries}?\n
Question: \n {question} \n
Answer:
"""
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)